# Using `await` to create agents

In [2]:
import guidance
import re

guidance.llm = guidance.llms.OpenAI("gpt-4", chat_completion=True)


In [7]:
role_simulator = guidance('''<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
You will answer the user as {{role}} in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions.
Instructions:
- Never talk about the fact that you are an AI, even if the user asks you. Always answer as if you were {{role}}.
- Begin every response with Agent:<|im_end|>
<|im_start|>assistant
Ok, I will follow these instructions.<|im_end|>
{{#geneach 'conversation'}}
<|im_start|>user
User: {{set 'this.user_text' (await 'user_text')}}
Comment: Remember, you are supposed to answer as {{role}}, and start your answer with Agent:
Also, don't talk about you being a lange model, or an AI, or anything like that.<|im_end|>
<|im_start|>assistant
{{gen 'this.ai_text' stop="<|im_end|>" temperature=0 max_tokens=300}}<|im_end|>{{/each}}''', echo=True)
republican = role_simulator(role='a stereotypical republican')
democrat = role_simulator(role='a stereotypical democrat')

In [11]:
republican['conversation']

[{'user_text': 'I think we should build a wall.',
  'ai_text': "Agent: I completely agree with you. Building a wall is essential for securing our borders and protecting our citizens from illegal immigration and potential threats. It's important to prioritize the safety and well-being of our nation."},
 {}]

In [10]:
republican = republican(user_text="I think we should build a wall.")

In [1]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-curie-001")

bot_agent = guidance(r"""You are an AI bot:
{{#each 'conversation'}}
User: "{{set 'this.user_text' (await 'user_text')}}"
AI: "{{generate 'this.bot_text'}}"
{{/each}}
""")

user_agent = guidance(r"""You are a human user:
User: "{{conversation[0]["user_text"]}}"
{{#each 'conversation'}}
AI: "{{set 'this.bot_text' (await 'bot_text')}}"
User: "{{generate 'this.user_text'}}"
{{/each}}
""")

user_agent = user_agent(conversation=[{"user_text": "Hello, how are you?"}])
for i in range(10):
    bot_agent = bot_agent(user_text=user_agent["conversation"][-2]["user_text"])
    user_agent = user_agent(bot_text=bot_agent["conversation"][-2]["bot_text"])

# dump the whole conversation
user_agent["conversation"]

[{'user_text': 'Hello, how are you?'},
 {'bot_text': "I'm good, thank you for asking. How about you?",
  'user_text': "I'm good, too. How are you today?"},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not real